In [ ]:
#https://indicnlp.ai4bharat.org/indicft/


In [1]:
#paths required
datafolder_p = r'D:\18-DS\github\SDSHL\data\processed'
datafolder_e = r'D:\18-DS\github\SDSHL\data\external'
datafolder_i = r'D:\18-DS\github\SDSHL\data\internal'
modelfolder  = r'D:\18-DS\data\models'
resultsfolder = r'D:\18-DS\github\SDSHL\data\results'

modelfolder_ft       = modelfolder + r'\fasttext_wiki.hi'
modelfolder_ft_ind   = modelfolder + r'\fasttext_indicnlp.hi'
modelfolder_ft_local   = modelfolder + r'\fasttext_local'


In [2]:
#fasttax pretrained Hindi binary/text format vectors 
#Common Crawl and Wikipedia 
cc300 = r'\cc.hi.300.bin'
cc100 = r'\cc.hi.100.bin'

#wiki
wiki100    = r'\wiki.hi.100.bin'
wiki300    = r'\wiki.hi.300.bin'
wiki300_vec= r'\wiki.hi.300.vec'

#indicnlp
indicnlp300     = r'\indicnlp.ft.hi.300.bin'
indicnlp300_vec = r'\indicnlp.ft.hi.300.vec'

# FastText Model donot need dataset but filename in a specific fasttext format for training
train_file = datafolder_p+ r'\3-train_ft.csv'
test_file = datafolder_p+ r'\3-test_ft.csv'

# to predict the result with ID we need this file. Because fasttext format doesnot have ID
#train_file1 = datafolder_p+ r'\2-train.csv'
test_file1 = datafolder_p+ r'\2-test.csv'

#This file is needed to create fasttext sentence embedding of full dataset 
filepath_fulldata = datafolder_p+ r'\2-Hinglish_Sarcasm_Clean.csv'

In [ ]:
# import pandas as pd

# file = datafolder_p + r'\2-Hinglish_Sarcasm_Clean.csv'
# df = pd.read_csv(file, sep='\t', index_col="ID")
# idx = list(df.index)

# file =  datafolder_p+ r'\2-train.csv' 
# df = pd.read_csv(file, sep='\t', index_col="ID")
# idx_train = df.index

# file =  datafolder_p+ r'\2-test.csv'
# df = pd.read_csv(file, sep='\t', index_col="ID")
# idx_test = df.index

# df = pd.read_csv(file_data, sep='\t', header=None)
# df['ID']=idx
# df=df.set_index('ID')

In [ ]:
#!pip install keras
#!pip install pytorch
#!pip install tensorflow

In [3]:
!pip install fasttext

     ---------------------------------------- 68.8/68.8 KB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pybind11-2.9.1-py2.py3-none-any.whl (211 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-win_amd64.whl size=245583 sha256=27b9eb2f96d76054ccb5a24c606f476f2cfc7571cb1134cbb25a2cffa7b56445
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\4e\ca\bf\b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [5]:
!pip install nltk

     ---------------------------------------- 1.5/1.5 MB 4.0 MB/s eta 0:00:00
     -------------------------------------- 273.8/273.8 KB 2.8 MB/s eta 0:00:00


In [6]:
#from __future__ import print_function

import pandas as pd
import nltk
import numpy as np
import re
import sklearn.metrics as metrics

#!pip uninstall fasttext
import fasttext
from fasttext import load_model

In [15]:
#nltk.download('gutenberg')
#nltk.download('genesis')
#nltk.download('inaugural')
#nltk.download('brown')
#nltk.download('webtext')
#nltk.download('nps_chat')
nltk.download('treebank')


[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\treebank.zip.


True

In [16]:
#C:\Users\admin\AppData\Roaming\nltk_data.
from nltk.book import nps_chat

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [ ]:
df =pd.read_csv(train_file, index_col=None, header=None, sep='\t')
corpus = list(df[1])
tokenized_corpus = [nltk.word_tokenize(doc) for doc in corpus]
df['sentence_tkn'] = tokenized_corpus

#This is required to create word embedding of full dataset
df_full =pd.read_csv(filepath_fulldata, usecols=['ID','label','sentence'], sep="\t", index_col="ID")
corpus_full = list(df_full['sentence'])
tokenized_corpus_full = [nltk.word_tokenize(doc) for doc in corpus_full]
df_full['sentence_tkn'] = tokenized_corpus_full

In [5]:
# #Load 2-train data in memory. This file is required for lookup and creating wordvec for each sentence.
# df =pd.read_csv(train_file1, usecols=['ID','label','sentence'], sep="\t", index_col="ID")
# corpus = list(df['sentence'])
# tokenized_corpus = [nltk.word_tokenize(doc) for doc in corpus]
# df['sentence_tkn'] = tokenized_corpus

df_test=pd.read_csv(test_file1, usecols=['ID','label','sentence'], sep="\t", index_col="ID")
y_test =np.array(df_test['label'])
X_test=df_test['sentence']

# Some Common Functions

# Language Identification using fasttext

In [ ]:
ftmodel_lang = fasttext.load_model(vectorfolder+'\lid.176.bin')

In [ ]:
# print('\n',len(ftmodel_lang.words) )
# print('\n',ftmodel_lang.get_dimension() )
# #print(ftmodel_lang.get_labels())
# print('\n',ftmodel_lang.get_line("hari"))
# print('\n',ftmodel_lang.get_analogies('apple','orange','juice',10) )

# print ('\n', ftmodel_lang.get_subwords('Hari') )
# #print( ftmodel_lang.get_words())
# print ('\n',ftmodel_lang.get_sentence_vector('Hari Thapliyal'))
# print ('\n',ftmodel_lang.get_subword_id('apple') )
# print ('\n',ftmodel_lang.get_nearest_neighbors('apple') )
# ftmodel_lang.predict(['Thapliyal', 'हर'])
n=27
list(zip(tokenized_corpus[n],ftmodel_lang.predict(tokenized_corpus[n])[0],ftmodel_lang.predict(tokenized_corpus[n])[1]))

# Get Word Vector Without Pre-Training

In [28]:
# Set values for various parameters
feature_size = 15    # Word vector dimensionality  
window_context = 20  # Context window size                                                                                    
min_word_count = 1   # Minimum word count                        
sample = 1e-3        # Downsample setting for frequent words
sg = 1               # skip-gram model

#window_length = 200 # The amount of words we look at per example. Experiment with this.

In [29]:
def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.wv[word])
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
    return feature_vector

def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [30]:
##THIS STEP TAKES VERY LONG TIME

from gensim.models.fasttext import FastText
ft_model = FastText(tokenized_corpus, size=feature_size, 
                          window=window_context, min_count = min_word_count,
                          sg=sg, sample=sample, iter=5000)

In [31]:
#save the model
ft_model.save(modelfolder_ft_local + r'\model_fasttext_local.bin')

In [32]:
#ft_tnseEmbedded, ft_words, ft_wvs, ft_labels = getTNSE_Embedded(ft_model)

# get document level embeddings
ft_doc_features = averaged_word_vectorizer(corpus=tokenized_corpus_full, model=ft_model,
                                             num_features=feature_size)

ft_features = pd.DataFrame(ft_doc_features, index=df_full.index)
ft_features['label']=df_full.label
ft_features.to_csv(datafolder_p + r"\embedding_ft_local.csv")
ft_features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,label
ID,,,,,,,,,,,,,,,,
1,0.519313,-0.459539,-0.185066,-0.091893,-0.072511,0.104139,1.010676,0.215087,0.304260,0.229366,-0.538796,0.310136,0.081293,-0.582336,-0.925841,1
5,0.574375,-0.630665,0.596369,0.060031,-0.129327,-0.342102,0.216957,0.074616,0.059842,0.525639,-0.002986,-0.501358,0.095393,-0.271787,-0.727843,1
12,0.777627,-0.219662,0.097365,0.023982,0.381467,0.716699,0.789555,0.038210,0.388016,0.544673,1.197809,-0.248336,0.052653,-0.342404,-0.735357,1
13,0.011255,-1.342419,-0.876766,-0.867125,-0.507696,0.127758,0.722860,-0.580572,0.518736,1.038196,-0.055575,0.899874,0.274856,-1.229804,-1.066004,1
26,0.778493,-0.494959,0.170979,0.832334,0.467761,0.578545,0.875711,1.520114,0.190859,1.135316,0.025323,0.009384,0.875736,0.001797,-1.662741,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9347,0.983233,0.231945,-0.255089,-0.622469,0.352465,-0.222344,0.626213,0.343675,0.212641,0.593361,-0.027918,-0.010977,0.177415,-0.729206,-0.510179,1
9348,1.446260,-0.843464,-0.095849,0.620362,0.266516,-0.066601,1.194153,0.330961,0.219292,-0.043578,-0.200063,-0.948163,0.223242,-1.490455,-0.503475,1
9349,0.732688,-0.808272,-0.517679,1.349131,0.413501,0.012112,1.095162,0.215480,0.958502,1.374768,-0.172372,-0.452244,0.126437,-0.583281,-0.950802,1


# Using Pretrained Vector

### How to compress large size pretrained binary model

In [ ]:
#load cc.hi.300.bin 
ft300 = fasttext.load_model(modelfolder_ft+ r'\cc.hi.300.bin')
ft300.get_dimension()

#reduce dimension from 300 to 100
import fasttext.util
fasttext.util.reduce_model(ft300, 100)
ft300.get_dimension()

#save the model with reduced dimension
ft300.save_model(vectorfolder+'\cc.hi.100.bin')

### Method 1: Load a pretrained model in memory

In [ ]:
#ftmodel_cc300 = fasttext.load_model(modelfolder_ft+cc300)

In [ ]:
print('Model file name:'+modelfolder_ft+cc300)
#print (ftmodel_cc300.get_dimension(), len(ftmodel_cc300.words))

In [8]:
ftmodel_cc100 = fasttext.load_model(modelfolder_ft+cc100)

In [ ]:
print('file name:'+modelfolder_ft+cc100)
#print (ftmodel_cc100.get_dimension(), len(ftmodel_cc100.words))

In [9]:
ftmodel_indnlp300 = fasttext.load_model(modelfolder_ft_ind+indicnlp300)

In [ ]:
print('Model file name:'+modelfolder_ft_ind+indicnlp300)
#print (ftmodel_indnlp300.get_dimension(), len(ftmodel_indnlp300.words))

### Method 3: Read details of Pretrained Vector of Text format

In [ ]:
#fasttest wiki indicnlp vector

In [19]:
import io
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

dict_ftvec=load_vectors(modelfolder_ft+wiki300_vec)
print ( len(dict_ftvec))

158016


### Method 4: How to create model from pretrained-vector

### <font color=red>wiki300_vec<font>

In [8]:
#used pretrained vector to get the word embedding. This model can also be used for prediction
#df should be in fastText format
print (train_file)
ftmodel_wiki300_vec = fasttext.train_supervised(train_file,dim=300, \
                                     lr=0.5, epoch=25, wordNgrams=2, bucket=200000,\
                                     pretrainedVectors=modelfolder_ft + wiki300_vec)
#loss='ova'

D:\18-DS\github\SDSHL\data\processed\3-train_ft.csv


In [9]:
len(ftmodel_wiki300_vec.words)

160445

In [10]:
print ( 'wiki300_vec has 158016 words - Method 3')
print ( 'ftmodel_wiki300_vec - a finetuned model from wiki300_vec vector has  160659 word - Method 4')
print ( 'ftmodel_vec300 model is created from existing vector')
#It means 160659-158016 are the new words in the fine tuned model as done above.

wiki300_vec has 158016 words - Method 3
ftmodel_wiki300_vec - a finetuned model from wiki300_vec vector has  160659 word - Method 4
ftmodel_vec300 model is created from existing vector


### <font color=red>indicnlp300_vec<font>

In [11]:
#used pretrained vector to get the word embedding. This model can also be used for prediction
#df should be in fastText format

ftmodel_indicnlp300_vec = fasttext.train_supervised(train_file,dim=300, \
                                     lr=0.5, epoch=25, wordNgrams=2, bucket=200000,\
                                     pretrainedVectors=modelfolder_ft_ind + indicnlp300_vec)
#loss='ova'

In [12]:
len(ftmodel_indicnlp300_vec.words)

328394

####  \****Indicnlp based model has more words than wiki based model\****

# Performance of wiki_pretrained model

In [13]:
prediction={}

#### ftmodel_wiki300_vec

In [14]:
ftmodel_wiki300_vec.test(train_file)

(1800, 0.9994444444444445, 0.9994444444444445)

In [16]:
def pred_for_test(model):
    y_pred=[]
    for i in range(len(X_test)):
        p=model.predict(X_test.iloc[i])
        x1 = int(p[0][-1][-1])
        x2 = p[1][0]
        if x1==0: x2= 1-x2
        y_pred.append( x2 )
    return y_pred    


def print_matrics(y_pred):
    threshold=0.5
    y_pred1=[]
    for i in y_pred :
        if i>threshold:
            y_pred1.append(1)
        else:
            y_pred1.append(0)

    acc = np.round( metrics.accuracy_score(y_test,y_pred1), 2)
    recall = np.round( metrics.recall_score(y_test,y_pred1), 2)
    precision = np.round( metrics.precision_score(y_test,y_pred1), 2)
    f1 = np.round( metrics.f1_score(y_test,y_pred1), 2)
    roc = np.round( metrics.roc_auc_score(y_test,y_pred1), 2)

    print("Accuracy : ", acc )
    print("Recall   : ", recall )
    print("Precision: ", precision )
    print("F1       : ", f1 )
    print("ROC      : ", roc )

In [17]:
y_pred = pred_for_test(ftmodel_wiki300_vec)
prediction['fasttext_wiki_pretrained'] = y_pred
print_matrics(y_pred)

Accuracy :  0.72
Recall   :  0.66
Precision:  0.75
F1       :  0.7
ROC      :  0.72


#### ftmodel_indicnlp300_vec

In [18]:
ftmodel_indicnlp300_vec.test(train_file)

(1800, 0.9994444444444445, 0.9994444444444445)

In [19]:
ftmodel_indicnlp300_vec.test(test_file)

(200, 0.7, 0.7)

In [20]:
y_pred = pred_for_test(ftmodel_indicnlp300_vec)
prediction['fasttext_indicnlp_pretrained'] = y_pred
print_matrics(y_pred)

Accuracy :  0.7
Recall   :  0.63
Precision:  0.73
F1       :  0.68
ROC      :  0.7


In [34]:
df_prediction= pd.DataFrame(prediction, index=df_test.index)
df_prediction.to_csv(resultsfolder+ r'\model_predictions_fasttext.csv')
df_prediction

,fasttext_wiki_pretrained,fasttext_indicnlp_pretrained
ID,,
5212,0.109270,0.043942
8028,0.960893,0.968121
2364,0.032174,0.017442
5805,0.000468,0.000220
5236,0.004674,0.008743
...,...,...
7171,-0.000009,-0.000010
8819,0.540725,0.451919
2686,0.023583,0.025390


# Convert Sentences to Vector & save Embedding

In [ ]:
#Use ftmodel_cc100 to generate senetence vector. This vector later on can be \
#used in classical ML models for classification.

In [22]:
def df_to_vec(n_features,df,model ):

    sent_all_vec = np.zeros((len(df), n_features), dtype='float32')
    i=0
    for row in np.array(df):
        sent_vec=np.zeros(n_features)
        j=0
        for word in row:
            sent_vec=sent_vec+ model.get_word_vector(word).astype('float32') 
            j+=1
        sent_vec = sent_vec/j
        sent_all_vec[i]=sent_vec
        i+=1
    return sent_all_vec

#### ftmodel_wiki300_vec

In [23]:
#convert sentence to vector using pretrained fasttext
df1 = pd.DataFrame( df_to_vec(300, df_full['sentence_tkn'], ftmodel_wiki300_vec), index=df_full.index)
df1['label'] = df_full['label']

In [24]:
df1.to_csv(datafolder_p+r'\embedding_ft300_wiki_pretrained.csv')

#### ftmodel_indicnlp300_vec

In [25]:
#convert sentence to vector using pretrained fasttext
df1 = pd.DataFrame( df_to_vec(300, df_full['sentence_tkn'], ftmodel_indicnlp300_vec), index=df_full.index)
df1['label'] = df_full['label']

In [26]:
df1.to_csv(datafolder_p+r'\embedding_ft300_indicnlp_pretrained.csv')

In [27]:
df1

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,label
ID,,,,,,,,,,,,,,,,,,,,,
1,0.047358,-0.065001,0.135721,0.088405,0.040134,0.165502,-0.155682,-0.095986,0.154054,0.057132,...,-0.114018,0.002591,0.076288,-0.083610,0.009277,-0.208232,0.033195,0.017050,-0.129891,1
5,0.091848,0.030187,0.076497,0.099425,0.062724,0.128292,-0.106851,-0.120260,0.114149,0.073124,...,-0.052836,0.074878,0.056161,-0.067253,-0.036870,-0.174462,0.000053,-0.006892,-0.095432,1
12,0.054877,-0.122250,0.005790,0.014169,0.007954,0.108095,-0.175122,-0.036322,0.083358,0.054436,...,-0.224565,0.113609,-0.028683,-0.058093,-0.024254,-0.043670,0.076704,0.026717,-0.104058,1
13,0.086249,0.014710,0.024039,0.072895,-0.000357,0.211339,-0.112617,-0.085320,0.079474,0.093073,...,-0.076949,0.111832,-0.035515,0.039484,0.060837,-0.161844,0.072351,-0.024754,-0.071523,1
26,0.039130,-0.115447,0.105212,0.044516,0.024228,0.133685,-0.103824,-0.073412,0.072047,0.034901,...,-0.076331,0.027883,0.007058,-0.046341,0.046777,-0.281825,-0.022362,0.033455,-0.082226,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9347,0.002272,-0.086058,-0.038732,0.153019,-0.074544,0.136648,-0.167548,-0.089112,-0.003029,0.065542,...,-0.094834,0.119982,0.036304,0.022565,-0.061524,-0.109919,0.048014,0.096063,-0.084558,1
9348,0.122960,-0.072445,0.117342,0.054222,-0.055762,0.079046,-0.172321,-0.165351,0.070567,0.102010,...,-0.149850,0.052122,0.129005,-0.117767,-0.056222,-0.093479,-0.070339,0.053621,-0.130905,1
9349,0.043652,-0.038357,0.042601,0.095946,-0.007229,0.077493,-0.083869,-0.072260,0.015455,0.051727,...,-0.169764,0.075684,0.048517,-0.089417,0.019201,-0.072376,0.007109,0.018361,-0.046572,1


# Unsupervised Learning

### Method 5: How to cerate a new model using our corpus. This can be used as a pretrained model for other NLP tasks

In [ ]:
import fasttext
# Skipgram model :
ftmodel_usup = fasttext.train_unsupervised(train_datafile, model='skipgram', dim=300, minn=2, maxn=5,\
                                     lr=0.5, epoch=5, wordNgrams=2, bucket=200000)

# or, cbow model :
#model = fasttext.train_unsupervised('data.txt', model='cbow')

In [ ]:
print( len(model.words)) 
#print (model.words)
ftmodel_usup.save_model("sdsms.bin")

In [ ]:
#load the saved & trained model.
ftmodel_usup = fasttext.load_model("sdsms.bin")

In [ ]:
print(ftmodel_usup.get_dimension())
print(ftmodel_usup.get_words())

In [ ]:
ft_doc_features = averaged_word_vectorizer(corpus=tokenized_corpus, model=ftmodel_usup,
                                             num_features=feature_size)

In [ ]:
ftmodel_vec300.predict('हिंसा प्रति हिंसा को रोकना दुरुह कार्य है.')

# <font color=red> Experiments

In [ ]:
# import io
# vecfile_name= vectorfolder + '\wiki.hi.vec'

# fin = io.open(vecfile_name, 'r', encoding='utf-8', newline='\n', errors='ignore')
# data = {}
# for line in fin:
#     tokens = line.rstrip().split(' ')
#     data[tokens[0]] = map(float, tokens[1:])

In [ ]:
# for a in data["माता"]:
#     print (a)

In [ ]:
import fasttext
#help(fasttext.FastText)

In [ ]:
from fasttext import FastText

In [ ]:
#help(FastText)

In [ ]:
import fasttext


In [ ]:
# test.get_line('I am hari')

# test.get_sentence_vector('Hari Thapliyal')

#test.get_input_vector(1000000)

#test.get_subwords('Hari Thapliyal')

test.get_label_id('orange')


In [ ]:
# print('\n',len(test.words) )
# print('\n',test.get_dimension() )
# #print(test.get_labels())
# print('\n',test.get_line("hari"))
# print('\n',test.get_analogies('apple','orange','juice',10) )

# print ('\n', test.get_subwords('Hari') )
# #print( test.get_words())
# print ('\n',test.get_sentence_vector('Hari Thapliyal'))
# print ('\n',test.get_subword_id('apple') )
# print ('\n',test.get_nearest_neighbors('apple') )
# test.predict(['Thapliyal', 'हर'])
n=1000
list(zip(tokenized_corpus[n],test.predict(tokenized_corpus[n])[0],test.predict(tokenized_corpus[n])[1]))

In [ ]:
import numpy as np
a = np.array(tokenized_corpus)
a.reshape()

In [ ]:
b=[]
for c  in a:
    b.append( " ".join(c))
    

In [ ]:
a=set("".join(b).split(" "))

In [ ]:
print(a)

In [ ]:
len(tokenized_corpus)
tokenized_corpus[0]

In [ ]:
b=np.array(a)
b.shape
c= set( list(b))

In [ ]:
#fasttext.FastText(tokenized_corpus)

test= supervised(tokenized_corpus, size=feature_size, 
                              window=window_context, min_count = min_word_count,
                              sg=sg, sample=sample, iter=5000)

In [ ]:
datetime.sys()